In [1]:
import dspy
from dspy.retrieve.chromadb_rm import ChromadbRM
import os
from get_embedding_function import get_embedding_function
from populate_database import setup_database
from dspy.teleprompt import BootstrapFewShot
import pandas as pd
from dspy.datasets.dataset import Dataset
from dspy.evaluate.evaluate import Evaluate

/home/thuva_siva29/green-software-foundation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-12 16:21:52.638051: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-12 16:21:52.649554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 16:21:52.665508: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered

In [2]:
TRAINING_DATA = "queries_judgements.csv"

class CSVDataset(Dataset):
    def __init__(self, file_path, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
        df = pd.read_csv(file_path)
        self._train = df.iloc[0:15].to_dict(orient='records')

        self._dev = df.iloc[15:45].to_dict(orient='records')

dataset = CSVDataset(TRAINING_DATA)

trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

print(trainset)


[Example({'question': 'Is there any mention of optimizing storage utilization?', 'answer': 'Yes'}) (input_keys={'question'}), Example({'question': 'Is there any mention of adopting serverless cloud services?', 'answer': 'No'}) (input_keys={'question'}), Example({'question': 'Is there any mention of allowing independent scaling of logical components?', 'answer': 'Yes'}) (input_keys={'question'}), Example({'question': 'Is there any mention of scaling down Kubernetes applications when not in use?', 'answer': 'Not Applicable'}) (input_keys={'question'}), Example({'question': 'Is there any mention of reducing transmitted data?', 'answer': 'No'}) (input_keys={'question'}), Example({'question': 'Is there any mention of optimizing average CPU utilization?', 'answer': 'No'}) (input_keys={'question'}), Example({'question': 'Is there any mention of managing peak CPU utilization?', 'answer': 'No'}) (input_keys={'question'}), Example({'question': 'Is there any mention of scaling infrastructure base

In [3]:
CHROMA_PATH = os.getenv("CHROMA_PATH")
DOCUMENT_PATH="./documents/4.pdf"

emb_local = True

setup_database(DOCUMENT_PATH, True, emb_local, True)

embedder, collection_name = get_embedding_function(run_local=emb_local)

retriever_model = ChromadbRM(
    collection_name,
    CHROMA_PATH,
    embedding_function=embedder.embed_documents,
    k=5
)

lm = dspy.OllamaLocal(model='phi3')

dspy.settings.configure(lm=lm, rm=retriever_model)

✨  Database Cleared
Number of existing documents in DB: 0
👉 Adding new documents: 34


In [4]:
retriever = dspy.Retrieve(k=3)
print(retriever("Is there any mention of optimizing the size of AI/ML models to save storage space and reduce memory usage during inference").passages)

['When you load the front page you see multiple rows of different kinds of movies. Netflix personalizes this\ndata and decides what kind of rows or what kind of movies should be displayed to a specific user. This\ndata is based on the user’s historical data and preferences.\n\nAlso, for that specific user, Netflix performs sorting of the movies and calculates the relevance ranking (for\nthe recommendation) of these movies available on their platform. In Netflix, Apache Spark is used for\ncontent recommendations and personalization.\n\nA majority of the machine learning pipelines are run on these large spark clusters. These pipelines are\nthen used to do row selection, sorting, title relevance ranking, and artwork personalization among others.', 'Hystrix library is designed to do this job. It helps you to control the interactions between these distributed\nservices by adding latency tolerance and fault tolerance logic. Hystrix does this by isolating points of\naccess between the service

In [5]:
class GenerateAnswer(dspy.Signature):
    """Give a concrete yes or no or not applicable answer to a question based on the retrieved context."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="Output nothing else apart from either yes, no, or not applicable.")

class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [6]:
result = RAG().forward("Is there any mention of optimizing the size of AI/ML models to save storage space and reduce memory usage during inference?")
print(result.answer)

Not applicable.


In [8]:
print(lm.inspect_history(n=1))





Give a concrete yes or no or not applicable answer to a question based on the retrieved context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: Output nothing else apart from either yes, no, or not applicable.

---

Context:
[1] «switches between formats (for the best viewing experience) when it’s needed.
User data is saved in AWS such as searches, viewing, location, device, reviews, and likes, Netflix uses it
to build the movie recommendation for users using the Machine learning model or Hadoop.»
[2] «3.2. How Netflix balance the high traffic load
1. Elastic Load Balancer
elastic-load-balancer

ELB in Netflix is responsible for routing the traffic to front-end services. ELB performs a two-tier
load-balancing scheme where the load is balanced over zones first and then instances (servers).»
[3] «When the user loads the Netflix app on his/her device 

In [7]:
teleprompter = BootstrapFewShot(metric=dspy.evaluate.answer_exact_match)

compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

evaluate_on_dataset = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=30)

#Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match
evaluate_on_dataset(compiled_rag, metric=metric)

 47%|███████████████████████████████████▍                                        | 7/15 [04:14<04:50, 36.31s/it]


Bootstrapped 4 full traces after 8 examples in round 0.


Average Metric: 15 / 30  (50.0): 100%|██████████████████████████████████████████| 30/30 [23:09<00:00, 46.32s/it]
/home/thuva_siva29/green-software-foundation/.venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' '✔️ [True]' 'False' 'False' 'False' 'False' '✔️ [True]' 'False'
 'False' '✔️ [True]' 'False' '✔️ [True]' 'False' 'False' '✔️ [True]'
 'False' 'False' '✔️ [True]' '✔️ [True]' '✔️ [True]' '✔️ [True]' 'False'
 '✔️ [True]' 'False' '✔️ [True]' '✔️ [True]' '✔️ [True]' '✔️ [True]'
 'False' '✔️ [True]']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


Average Metric: 15 / 30  (50.0%)


,question,example_answer,context,pred_answer,answer_exact_match
0,Is there any mention of regularly scanning for and fixing vulnerabilities?,No,"['When the user loads the Netflix app on his/her device firstly AWS instances come into the picture and\nhandle some tasks such as login, recommendations, search,...","No, based on the provided context, there is no explicit mention of Netflix's regular practices or procedures related to scanning for and fixing vulnerabilities in...",False
1,Is there any mention of evaluating other CPU architectures?,No,"['When the user loads the Netflix app on his/her device firstly AWS instances come into the picture and\nhandle some tasks such as login, recommendations, search,...",No,✔️ [True]
2,Is there any mention of compressing data transmitted over networks?,No,['3.2. How Netflix balance the high traffic load\n1. Elastic Load Balancer\nelastic-load-balancer\n\nELB in Netflix is responsible for routing the traffic to front-end services. ELB performs a...,"No, there is no explicit mention of compressing data transmitted over networks in the provided context. The text focuses on load balancing using Elastic Load...",False
3,Is there any mention of compressing stored data?,Yes,"['When the user loads the Netflix app on his/her device firstly AWS instances come into the picture and\nhandle some tasks such as login, recommendations, search,...","No, the provided context does not mention any specific strategies related to compressing stored data in order to directly manage peak CPU utilization at Netflix....",False
4,Is there any mention of using DDoS protection?,No,"['When the user loads the Netflix app on his/her device firstly AWS instances come into the picture and\nhandle some tasks such as login, recommendations, search,...","No, based on the provided context, there is no explicit mention of using DDoS protection. The information given focuses primarily on how Netflix handles high...",False
5,Is there any mention of scheduling Kubernetes cron jobs during off-peak hours?,Not Applicable,"['switches between formats (for the best viewing experience) when it’s needed.\nUser data is saved in AWS such as searches, viewing, location, device, reviews, and likes,...",No,False
6,Is there any mention of choosing server regions that are closest to users?,Yes,['3.2. How Netflix balance the high traffic load\n1. Elastic Load Balancer\nelastic-load-balancer\n\nELB in Netflix is responsible for routing the traffic to front-end services. ELB performs a...,Yes,✔️ [True]
7,Is there any mention of prioritizing critical traffic and shedding lower priority traffic?,Not Applicable,"['switches between formats (for the best viewing experience) when it’s needed.\nUser data is saved in AWS such as searches, viewing, location, device, reviews, and likes,...",No,False
8,Is there any mention of implementing circuit breaker patterns?,Not Applicable,['3.2. How Netflix balance the high traffic load\n1. Elastic Load Balancer\nelastic-load-balancer\n\nELB in Netflix is responsible for routing the traffic to front-end services. ELB performs a...,No,False
9,Is there any mention of adopting serverless cloud services to optimize resource usage?,No,"['switches between formats (for the best viewing experience) when it’s needed.\nUser data is saved in AWS such as searches, viewing, location, device, reviews, and likes,...",No,✔️ [True]


50.0

In [8]:
compiled_rag.save("final_trained_model.json")